
# Dog's Breed Identification

![Dogs](https://s.wsj.net/public/resources/images/B3-EU419_201908_GR_20190822110317.jpg)

* Generate dataset
* Build a model
* Predict

### Importing Necessary Libraries

In [ ]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator as Imgen

from keras.models import Sequential,load_model
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,GlobalAveragePooling2D,Dropout

from keras.preprocessing import image

import cv2

import pickle

**Reading the data and adding the extension to it.**

In [ ]:
labels = pd.read_csv("../input/dog-breed-identification/labels.csv")
sample = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')
labels.head()

In [ ]:
def app_jpg(id):
    return id+".jpg"

In [ ]:
labels['id'] = labels['id'].apply(app_jpg)
sample['id'] = sample['id'].apply(app_jpg)

**Data Generator**

In [ ]:
datagen = Imgen(preprocessing_function=keras.applications.nasnet.preprocess_input,
                  shear_range=0.2,
                  zoom_range=0.2,
                  horizontal_flip=True,
                  validation_split=0.1
                 )

In [ ]:
train_ds = datagen.flow_from_dataframe(
    labels,
    directory = '../input/dog-breed-identification/train', 
    x_col = 'id',
    y_col = 'breed',
    subset="training",
    color_mode="rgb",
    target_size = (331,331),
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=123,
)


val_ds = datagen.flow_from_dataframe(
    labels,
    directory = '../input/dog-breed-identification/train',
    x_col = 'id',
    y_col = 'breed',
    subset="validation",
    color_mode="rgb",
    target_size = (331,331),
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=123,
)

**Getting class names**

In [ ]:
a = train_ds.class_indices
class_names = list(a.keys())
class_names[:10]

**Getting one batch of data to visualize**

In [ ]:
x,y = next(train_ds)
x.shape

**Function to plot images**

In [ ]:
def plot_images(img,labels):
    plt.figure(figsize=[15,10])
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.imshow(img[i])
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis('off')

In [ ]:
plot_images(x,y)

## Model

**Getting a Pre-Trained Model**

In [ ]:
from keras.applications.nasnet import NASNetLarge
from keras.applications.inception_resnet_v2 import InceptionResNetV2

In [ ]:
base_model = InceptionResNetV2(include_top=False,
                     weights='imagenet',
                     input_shape=(331,331,3)
                     )
base_model.trainable = False

**Defining our Model**

In [ ]:
model = Sequential([
    base_model,
    
    GlobalAveragePooling2D(),
    
    Dense(256,activation = 'relu'),
    Dropout(0.5),
    
    Dense(120,activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

**Defining callbacks to preserve the best results**

In [ ]:
my_calls = [keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=2),
            keras.callbacks.ModelCheckpoint("Model.h5",verbose=1,save_best_only=True)]

### Train

In [ ]:
hist = model.fit(train_ds,epochs=25,validation_data=val_ds,callbacks=my_calls)

**Plotting the losses and acuuracies**

In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
plt.plot(hist.epoch,hist.history['accuracy'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_accuracy'],label = 'validation')

plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(hist.epoch,hist.history['loss'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_loss'],label = 'validation')

plt.title("Loss")
plt.legend()
plt.show()


**Loading saved model**

In [ ]:
model = load_model("./Model.h5")

## Prediction

**Genrating test dataset**

In [ ]:
testgen = Imgen(preprocessing_function=keras.applications.inception_resnet_v2.preprocess_input)

In [ ]:
test_ds = testgen.flow_from_dataframe(
    sample,
    directory = '../input/dog-breed-identification/test',
    x_col = 'id',
    y_col = None,
    target_size = (331,331),
    class_mode= None,
    batch_size=32,
    shuffle=False
)

**Making Predictions**

In [ ]:
predictions = model.predict(test_ds,verbose=1)

- To get the most probable result

In [ ]:
pred  = [np.argmax(i) for i in predictions]

In [ ]:
X = next(test_ds)
X.shape

**Visualizing images against predictions**

In [ ]:
def check_images(img):
    plt.figure(figsize=[25,10])
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.imshow(img[i])
        plt.title(class_names[pred[i]])
        plt.axis('off')

In [ ]:
check_images(X)